# Array-Based Sequences

Adapted from Data Structures and Algorithms in Python

Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser

John Wiley & Sons, 2013

## 1. Dynamic Array

In [320]:
import ctypes

class DynamicArray:
    """A dynamic array class akin to a simplified Python list."""
    
    def __init__(self, da=None):
        """Create an empty array."""
        self._n = 0 # count acutal elements
        self._capacity = 8 # default array capacity
        self._A = self._make_array(self._capacity) # low-level array
        if da:
            self.extend(da)

    ### subscript-based access ###
    
    def _normalize_idx(self, k): # private utility
        nidx = k
        if nidx < 0:
            nidx += self._n
            if nidx < 0:
                nidx = 0
        return nidx
        
    def __getitem__(self, k):
        """Return element at index k."""
        assert(isinstance(k, int))
        k = self._normalize_idx(k)
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        return self._A[k] # retrieve from array
    
    def __setitem__(self, k, val):
        """Set sefl[k] = val."""
        assert(isinstance(k, int))
        k = self._normalize_idx(k)
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        self._A[k] = val
        
    def __delitem__(self, k):
        """Implement del self[k]."""
        assert(isinstance(k, int))
        k = self._normalize_idx(k)
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        for i in range(k, self._n - 1):
            self._A[i] = self._A[i+1] # shfit leftward to fill gap
        self._A[self._n - 1] = None # help garbage collection
        self._n -= 1
        self._shrink()
        
    ### stringification ###
    
    def __str__(self):
        """Return a list of element separated by comma."""
        if not self._n:
            return '[]'
        return '[' + ', '.join(str(e) for e in self.__iter__()) + ']'
    
    def __repr__(self):
        if not self._n:
            return '[]'
        return '[' + ', '.join(str(e) for e in self.__iter__()) + ']'
    
    ### single-element manipulation ###
    
    def append(self, obj):
        """Add object to end of the array."""
        if self._n == self._capacity: # not enough room
            self._resize(2 * self._capacity) # double capacity
        self._A[self._n] = obj
        self._n += 1
        
    def _resize(self, c): # private utility
        """Resize internal array to capacity c."""
        B = self._make_array(c) # new array
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c
        
    def _make_array(self, c):
        """Return new array with capacity c."""
        return (c * ctypes.py_object)()
    
    def insert(self, k, value):
        """Insert value at index k, shifting subsequent values rightward."""
        if self._n == self._capacity: # not enough room
            self._resize(2 * self._capacity)
        for j in range(self._n, k, -1):
            self._A[j] = self._A[j-1] # shift rightmost first
        self._A[k] = value
        self._n += 1
        
    def remove(self, value):
        """Remove first occurrence of value (or raise ValueError)."""
        for k in range(self._n):
            if self._A[k] == value:
                for j in range(k, self._n - 1):
                    self._A[j] = self._A[j+1] # shfit leftward to fill gap
                self._A[self._n - 1] = None # help garbage collection
                self._n -= 1
                self._shrink()
                return
        raise ValueError('value not found')
        
    def _shrink(self):
        """Shrink the array to its half capacity."""
        if (1.0 * self._n / self._capacity < 0.25 and self._capacity // 2 > 8):
            B = self._make_array(self._capacity // 2)
            for k in range(self._n):
                B[k] = self._A[k]
            self._A = B
            self._capacity = self._capacity // 2
            
    def pop(self, k=-1):
        """Delete and return the element at index k."""
        val = self._A[k]
        self.__delitem__(k)
        return val
         
    ### predicates ###
    
    def __eq__(self, other):
        """Return True if this array contains the same elements (in order) as other."""
        if not isinstance(other, DynamicArray):
            return False
        if len(other) != len(self):
            return False
        for i in range(self._n):
            if self[i] != other[i]:
                return False
        return True
    
    def __contains__(self, value):
        """Return True if value is found in the array. False otherwise."""
        for i in range(self._n):
            if self._A[i] == value:
                return True
        return False
    
    ### queries ###
    
    def __len__(self):
        """Return number of elements stored in the array."""
        return self._n
    
    def min(self):
        """Return the minimum value in the array."""
        if not self._n:
            return None
        min_val = self._A[0]
        for i in range(1, self._n):
            if self._A[i] < min_val:
                min_val = self._A[i]
        return min_val
    
    def max(self):
        """Return the minimum value in the array."""
        if not self._n:
            return None
        max_val = self._A[0]
        for i in range(1, self._n):
            if self._A[i] > max_val:
                max_val = self._A[i]
        return max_val
    
    def index(self, value, i = 0, j=None):
        """
        Return the index of the first instance of value encountered in the array
        between index i (inclusive) and j (exclusive). if j is not specified,
        search through the end of the array for value. If value is not in the array,
        raise a ValueError.
        """
        if not j:
            j = self._n
        else:
            j = self._normalize_idx(j)
        for i in range(i, j):
            if self._A[i] == value:
                return i
        raise ValueError
        
    def count(self, value):
        """Return the number of occurences of value in the array."""
        count = 0
        for i in range(self._n):
            if self._A[i] == value:
                count += 1
        return count

    ### bulk operations ###
    
    def __add__(self, other):
        """Returns a new array that contains values in this array by those of other."""
        assert(isinstance(other, DynamicArray))
        da = DynamicArray()
        for i in range(self._n):
            da.append(self._A[i])
        for i in range(len(other)):
            da.append(other[i])
        return da
    
    def clear(self):
        self.__init__()
        
    def copy(self):
        """Return a new array that contains the same values as this array."""
        da = DynamicArray()
        for i in range(self._n):
            da.append(self._A[i])
        return da
    
    def extend(self, other):
        """Adds all elements, in oroder , from other to this array."""
        for e in other:
            self.append(e)
    
    def __iter__(self):
        """Return a genarator object."""
        for i in range(self._n):
            yield self._A[i]

In [291]:
# Test subscript-based access
from unittest import TestCase
import random

tc = TestCase()
data = [1, 2, 3, 4]
da = DynamicArray(data)
    
for i in range(len(data)):
    tc.assertEqual(da[i], data[i])
    
with tc.assertRaises(IndexError):
    x = da[100]
    
with tc.assertRaises(IndexError):
    da[100] = 0
    
with tc.assertRaises(IndexError):
    del da[100]
    
da[1] = data[1] = 20
del data[0]
del da[0]

for i in range(len(data)):
    tc.assertEqual(da[i], data[i])
    
data = [random.randint(1, 100) for _ in range(100)]
da = DynamicArray()
for e in data:
    da.append(e)
for i in range(len(data)):
    da[i] = data[i] = random.randint(101, 200)
for i in range(50):
    to_del = random.randrange(len(data))
    del da[to_del]
    del data[to_del]

for i in range(len(data)):
    tc.assertEqual(da[i], data[i])
    
for i in range(0, -len(data), -1):
    tc.assertEqual(da[i], data[i])

In [283]:
# Test stringification
from unittest import TestCase
tc = TestCase()

da = DynamicArray()
tc.assertIsInstance(da, DynamicArray)
tc.assertEqual('[]', str(da))
tc.assertEqual('[]', repr(da))

da.append(1)
tc.assertEqual('[1]', str(da))
tc.assertEqual('[1]', repr(da))

for i in range(2, 5):
    da.append(i)
tc.assertEqual('[1, 2, 3, 4]', str(da))
tc.assertEqual('[1, 2, 3, 4]', repr(da))

In [292]:
# Test single-element manipulation
from unittest import TestCase
import random

tc = TestCase()
da = DynamicArray()
data = []

for _ in range(100):
    to_add = random.randrange(1000)
    data.append(to_add)
    da.append(to_add)
    
for i in range(100):
    tc.assertEqual(data[i], da[i])

for _ in range(100):
    to_in = random.randrange(1000)
    in_idx = random.randrange(len(data)+1)
    data.insert(in_idx, to_in)
    da.insert(in_idx, to_in)
    
for i in range(200):
    tc.assertEqual(data[i], da[i])
    
for _ in range(25):
    to_rem = data[random.randrange(len(data))]
    data.remove(to_rem)
    da.remove(to_rem)
    
for i in range(len(data)):
    tc.assertEqual(data[i], da[i])
    
with tc.assertRaises(ValueError):
    da.remove(9999)
   
for _ in range(10):
    pop_idx = random.randrange(len(data))
    data.pop(pop_idx)
    da.pop(pop_idx)
    
for i in range(len(data)):
    tc.assertEqual(data[i], da[i])

In [293]:
# Test predicates
from unittest import TestCase

tc = TestCase()
da1 = DynamicArray()
da2 = DynamicArray()

da1.append(1)
tc.assertNotEqual(da1, da2)

da2.append(1)
tc.assertEqual(da1, da2)

tc.assertFalse(2 in da1)
tc.assertFalse(3 in da2)

In [294]:
# Test queries
from unittest import TestCase
import random

tc = TestCase()
da = DynamicArray()

tc.assertEqual(0, len(da))
tc.assertEqual(0, da.count(1))
with tc.assertRaises(ValueError):
    da.index(1)
data = []    
for i in range(100):
    num = random.randrange(1000)
    da.append(num)
    data.append(num)
    
tc.assertEqual(100, len(da))
tc.assertEqual(min(data), da.min())
tc.assertEqual(max(data), da.max())
for e in data:
    tc.assertEqual(data.index(e), da.index(e))
    tc.assertEqual(data.count(e), da.count(e))
    
with tc.assertRaises(ValueError):
    da.index(1000)
    
data = [1, 2, 1, 2, 1, 1, 1, 2, 1]
da2 = DynamicArray()
for e in data:
    da2.append(e)
tc.assertEqual(1, da2.index(2))
tc.assertEqual(1, da2.index(2, 1))
tc.assertEqual(3, da2.index(2, 2))
tc.assertEqual(7, da2.index(2, 4))
tc.assertEqual(7, da2.index(2, 4, -1))
with tc.assertRaises(ValueError):
    da2.index(2, 4, -2)

In [295]:
# Test bulk operations
from unittest import TestCase
import random

tc = TestCase()
da1 = DynamicArray()
da2 = DynamicArray()
da3 = da1 + da2

tc.assertIsInstance(da3, DynamicArray)
tc.assertEqual(da1, da2)

data = [random.randrange(1000) for _ in range(50)]
data2 = [random.randrange(1000) for _ in range(50)]
for e in data:
    da1.append(e)
for e in data2:
    da2.append(e)
da3 = da1 + da2
tc.assertEqual(100, len(da3))

da1.clear()
tc.assertEqual(0, len(da1))

for _ in range(50):
    da1.append(random.randrange(1000))
da2 = da1.copy()
for i in range(len(da2)):
    tc.assertEqual(da2[i], da1[i])
    
da1.clear()
da1.extend(range(10))
da2.clear()
da2.extend(range(10))
for i in range(10):
    tc.assertEqual(da2[i], da1[i])

In [296]:
# Test iteration
from unittest import TestCase
import random

tc = TestCase()
da = DynamicArray()

data = [random.randrange(1000) for _ in range(100)]
da.extend(data)
tc.assertEqual(data, [e for e in da])

it1 = iter(da)
it2 = iter(da)
for e in data:
    tc.assertEqual(next(it1), e)
    tc.assertEqual(next(it2), e)

## 2. Sorting a Sequence

In [302]:
def insertion_sort(A):
    """Sort list of comparable elements into nondecreaing oder."""
    for i in range(1, len(A)):
        cur = A[i]
        j = i
        while j > 0 and A[j-1] > cur:
            A[j] = A[j-1]
            j -= 1
        A[j] = cur

In [319]:
# Test insertion_sort
from unittest import TestCase
import random

tc = TestCase()
data = [random.randrange(100) for _ in range(50)]
da = DynamicArray(data)
insertion_sort(da)
data.sort()
for i in range(len(data)):
    tc.assertEqual(data[i], da[i])

## 3. Storing High Scores for a Game

In [321]:
class GameEntry:
    """Represents one entry of a list of high scores."""
    
    def __init__(self, name, score):
        """Create an entry with given name and score."""
        self._name = name
        self._score = score
        
    def get_name(self):
        """Return the name of the person for this entry."""
        return self._name
    
    def get_score(self):
        """Return the score of this entry."""
        return self._score
    
    def __str__(self):
        """Return string representation of the entry."""
        return '({0}, {1})'.format(self._name, self._score) # e.g. '(Bob, 98)'

In [325]:
class Scoreboard:
    """Fixed-length sequence of high scores in nondecreasing order."""
    
    def __init__(self, capacity=10):
        """Initialize scoreboard with given maximum capacity."""
        self._board = [None] * capacity
        self._n = 0 # number of actual entries
        
    def __getitem__(self, k):
        """Return entry at index k."""
        return self._board[k]
    
    def __str__(self):
        """Return string representation of the high score list."""
        return '\n'.join(str(self._board[j]) for j in range(self._n))
    
    def add(self, entry):
        """Consider adding entry to high scores."""
        score = entry.get_score()
        
        # If the board not full or score is higher than last entry
        good = self._n < len(self._board) or score > self._board[-1].get_score()
        
        if good:
            if self._n < len(self._board): # no score drops from list
                self._n += 1
            # shift lower scores rightward to make room for new entry
            j = self._n - 1
            while j > 0 and self._board[j-1].get_score() < score:
                self._board[j] = self._board[j-1] # shift entry from j-1 to j
                j -= 1
            self._board[j] = entry           

In [343]:
if __name__ == '__main__':
    board = Scoreboard(5)
    for e in (('Rob', 750), ('Mike', 1105), ('Rose', 590), ('Jill', 740),
              ('Jack', 510), ('Anna', 660), ('Paul', 720), ('Bob', 1200)):
        ge = GameEntry(e[0], e[1])
        board.add(ge)
        print('After considering {0}, scoreboard is:'.format(ge))
        print(board)

After considering (Rob, 750), scoreboard is:
(Rob, 750)
After considering (Mike, 1105), scoreboard is:
(Mike, 1105)
(Rob, 750)
After considering (Rose, 590), scoreboard is:
(Mike, 1105)
(Rob, 750)
(Rose, 590)
After considering (Jill, 740), scoreboard is:
(Mike, 1105)
(Rob, 750)
(Jill, 740)
(Rose, 590)
After considering (Jack, 510), scoreboard is:
(Mike, 1105)
(Rob, 750)
(Jill, 740)
(Rose, 590)
(Jack, 510)
After considering (Anna, 660), scoreboard is:
(Mike, 1105)
(Rob, 750)
(Jill, 740)
(Anna, 660)
(Rose, 590)
After considering (Paul, 720), scoreboard is:
(Mike, 1105)
(Rob, 750)
(Jill, 740)
(Paul, 720)
(Anna, 660)
After considering (Bob, 1200), scoreboard is:
(Bob, 1200)
(Mike, 1105)
(Rob, 750)
(Jill, 740)
(Paul, 720)


## 4. Simple Cryptography

In [336]:
class CaesarCipher:
    """Class for doing encryption and decryption using a Caesar cipher."""
    
    def __init__(self, shift):
        """Construct Caesar cipher using given integer shift ro rotation."""
        encoder = [None] * 26
        decoder = [None] * 26
        for k in range(26):
            encoder[k] = chr((k + shift) % 26 + ord('A'))
            decoder[k] = chr((k - shift) % 26 + ord('A'))
        self._forward = ''.join(encoder)
        self._backward = ''.join(decoder)
        
    def encrypt(self, message):
        """Return string representing encripted message."""
        return self._transform(message, self._forward)
    
    def decrypt(self, secret):
        """Return decrypted message given entrypted secret."""
        return self._transform(secret, self._backward)
    
    def _transform(self, original, code):
        """Utility to perform transformation based on given code string."""
        msg = list(original)
        for k in range(len(msg)):
            if msg[k].isupper():
                j = ord(msg[k]) - ord('A')
                msg[k] = code[j]
        return ''.join(msg)

In [337]:
if __name__ == '__main__':
    cipher = CaesarCipher(3)
    message = "THE EAGLE IS IN PLAY; MEET AT JOE'S."
    coded = cipher.encrypt(message)
    print('Secret:', coded)
    answer = cipher.decrypt(coded)
    print('Message:', answer)

Secret: WKH HDJOH LV LQ SODB; PHHW DW MRH'V.
Message: THE EAGLE IS IN PLAY; MEET AT JOE'S.


## 5. Two-Dimensional Arrays and Positional Games

In [341]:
class TicTacToe:
    """Management of a Tic-Tac-Toe game."""
    
    def __init__(self):
        """Start a new game."""
        self._board = [[' '] * 3 for _ in range(3)]
        self._player = 'X'
    
    def mark(self, i, j):
        """Put an X or O mark at position (i, j) for next player's turn."""
        if not(0 <= i <=2 and 0 <= j <= 2):
            raise ValueError('Invalid board position')
        if self._board[i][j] != ' ':
            raise ValueError('Board position occupied')
        if self.winner() is not None:
            raise ValueError('Game is already complete')
        self._board[i][j] = self._player
        if self._player == 'X':
            self._player = 'O'
        else:
            self._player = 'X'
            
    def _is_win(self, mark):
        """Check whether the board configuration is a win for the given player."""
        board = self._board
        return (mark == board[0][0] == board[0][1] == board[0][2] or    # row 0
                mark == board[1][0] == board[1][1] == board[1][2] or    # row 1
                mark == board[2][0] == board[2][1] == board[2][2] or    # row 2
                mark == board[0][0] == board[1][0] == board[2][0] or    # column 0
                mark == board[0][1] == board[1][1] == board[2][1] or    # column 1
                mark == board[0][2] == board[1][2] == board[2][2] or    # column 2
                mark == board[0][0] == board[1][1] == board[2][2] or    # diagonal
                mark == board[0][2] == board[1][1] == board[2][0])
    
    def winner(self):
        """Return mark of winning player, or None to indicate a tie."""
        for mark in 'XO':
            if self._is_win(mark):
                return mark
        return None
    
    def __str__(self):
        """Return string representation of current game board."""
        rows = ['|'.join(self._board[r]) for r in range(3)]
        return '\n-----\n'.join(rows)

In [342]:
if __name__ == '__main__':
    game = TicTacToe()
    # X moves            # O moves:
    game.mark(1, 1);     game.mark(0, 2)
    game.mark(2, 2);     game.mark(0, 0)
    game.mark(0, 1);     game.mark(2, 1)
    game.mark(1, 2);     game.mark(1, 0)
    game.mark(2, 0)
    
    print(game)
    winner = game.winner()
    if winner is None:
        print('Tie')
    else:
        print(winner, 'wins')

O|X|O
-----
O|X|X
-----
X|O|X
Tie
